In [28]:
import numpy as np
import matplotlib
import matplotlib.pylab as plt
from sklearn import datasets, model_selection, ensemble, metrics
import xgboost as xgb
import shap

In [30]:
iris = datasets.load_iris()
data = np.c_[iris['data'], iris['target']]
X, y = data[:, :4], data[:, 4]
train_X, valid_X, train_y, valid_y = model_selection.train_test_split(X, y, test_size=0.2, random_state=42)
train_X.shape, valid_X.shape, train_y.shape, valid_y.shape

((120, 4), (30, 4), (120,), (30,))

In [33]:
model = xgb.XGBClassifier()
model.fit(train_X, train_y)

pred = model.predict(train_X)
print(metrics.confusion_matrix(train_y, pred))
print(metrics.accuracy_score(train_y, pred)) 

[[40  0  0]
 [ 0 41  0]
 [ 0  0 39]]
1.0


/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [34]:
# explain the model's predictions using SHAP values
# (same syntax works for LightGBM, CatBoost, and scikit-learn models)
explainer = shap.TreeExplainer(model)
shap_values = explainer.shap_values(train_X)

# visualize the training set predictions
shap.force_plot(explainer.expected_value, shap_values, train_X)

AssertionError: The shap_values arg looks looks multi output, try shap_values[i].